In [1]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import *
from openmmtools.states import *
from openmmtools.integrators import *
from simtk.openmm import CompoundIntegrator
from openmmtools import alchemy
from simtk import unit
from openmmtools.utils import RestorableOpenMMObject

import numpy as np
import copy, sys, time, os, math
import nglview
import logging
from simtk import openmm
import parmed
from openmmtools.utils import SubhookedABCMeta, Timer, RestorableOpenMMObject
from openmmtools import testsystems, alchemy
import netCDF4 as nc
from blues.integrators import AlchemicalExternalLangevinIntegrator
import mdtraj

finfo = np.finfo(np.float32)
rtol = finfo.precision
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logging.getLogger("openmmtools.alchemy").setLevel(logging.ERROR)
np.random.RandomState(seed=3134)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

/home/nathanlim/anaconda3/envs/openmmtools/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/nathanlim/anaconda3/envs/openmmtools/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Below try it inside classes

In [2]:
class DummySimulation(object):
    def __init__(self, integrator, system, topology):
        self.integrator = integrator
        self.topology = topology
        self.system = system

In [3]:
class NCMCMove(MetropolizedMove):
    
    def __init__(self, atom_subset=None, context_cache=None, n_steps=1000):
        self.n_steps = n_steps
        self.n_accepted = 0
        self.n_proposed = 0
        self.initial_energy = None
        self.initial_positions = None
        self.proposed_energy = None
        self.proposed_positions = None
        self.atom_subset = atom_subset
        self.context_cache = context_cache
        
    @property
    def statistics(self):
        """The acceptance statistics as a dictionary."""
        return dict(n_accepted=self.n_accepted, 
                    n_proposed=self.n_proposed,
                    initial_energy=self.initial_energy,
                    initial_positions=self.initial_positions,
                   proposed_energy=self.proposed_energy,
                   proposed_positions=self.proposed_positions)

    @statistics.setter
    def statistics(self, value):
        self.n_accepted = value['n_accepted']
        self.n_proposed = value['n_proposed']
        self.initial_energy = value['initial_energy']
        self.initial_positions = value['initial_positions']
        self.proposed_energy = value['proposed_energy']
        self.proposed_positions = value['proposed_positions']
        
    def _propose_positions(self, positions, center_of_mass):
        """Return new proposed positions.

        These method must be implemented in subclasses.

        Parameters
        ----------
        positions : nx3 numpy.ndarray
            The original positions of the subset of atoms that these move
            applied to.

        Returns
        -------
        proposed_positions : nx3 numpy.ndarray
            The new proposed positions.

        """
        
        print('Proposing positions...')
        
        reduced_pos = positions - center_of_mass
        # Define random rotational move on the ligand
        rand_quat = mdtraj.utils.uniform_quaternion(size=None, random_state=0)
        rand_rotation_matrix = mdtraj.utils.rotation_matrix_from_quaternion(rand_quat)
        #multiply lig coordinates by rot matrix and add back COM translation from origin
        proposed_positions = np.dot(reduced_pos, rand_rotation_matrix) * positions.unit + center_of_mass
        
        return proposed_positions
      
    
    def _create_dummy_simulation(self, integrator, system, topology):
        """
        Generate a dummy Simulation object because the Reporter
        expects an `openmm.Simulation` object in order to report information
        from it's respective attached integrator/
        """
        return DummySimulation(integrator, system, topology) 
    
    def _before_integration(self, context, integrator, thermodynamic_state):
        """Execute code after Context creation and before integration."""
        
        return self._create_dummy_simulation(integrator,
                                            thermodynamic_state.get_system(), 
                                            thermodynamic_state.topology)
    
    def _after_integration(self, context, thermodynamic_state):
        """Implement BaseIntegratorMove._after_integration()."""
        integrator = context.getIntegrator()
        try:
            # Accumulate acceptance statistics.
            ghmc_global_variables = {integrator.getGlobalVariableName(index): index
                                 for index in range(integrator.getNumGlobalVariables())}
            n_accepted = integrator.getGlobalVariable(ghmc_global_variables['naccept'])
            n_proposed = integrator.getGlobalVariable(ghmc_global_variables['ntrials'])
            self.n_accepted += n_accepted
            self.n_proposed += n_proposed
        except Exception as e:
            print(e)
            pass
        
    def _get_integrator(self, thermodynamic_state):
        return AlchemicalExternalLangevinIntegrator(
            alchemical_functions={ 'lambda_sterics':
                'min(1, (1/0.3)*abs(lambda-0.5))',
                'lambda_electrostatics':
                'step(0.2-lambda) - 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)'},
            splitting="H V R O R V H",
            temperature=thermodynamic_state.temperature,
            nsteps_neq=self.n_steps,
            timestep=4.0 * unit.femtoseconds,
            nprop=1,
            prop_lambda=0.3)
    
    def getMasses(self, atom_subset, topology):
        """
        Returns a list of masses of the specified ligand atoms.
        Parameters
        ----------
        topology: parmed.Topology
            ParmEd topology object containing atoms of the system.
        Returns
        -------
        masses: 1xn numpy.array * simtk.unit.dalton
            array of masses of len(self.atom_indices), denoting
            the masses of the atoms in self.atom_indices
        totalmass: float * simtk.unit.dalton
            The sum of the mass found in masses
        """
        atoms = [ list(topology.atoms())[i] for i in atom_subset]
        masses = unit.Quantity(np.zeros([int(len(atoms)), 1], np.float32), unit.dalton)
        for idx, atom in enumerate(atoms):
            masses[idx] = atom.element._mass
        totalmass = masses.sum()
        return masses, totalmass

    def getCenterOfMass(self, positions, masses):
        """Returns the calculated center of mass of the ligand as a numpy.array
        Parameters
        ----------
        positions: nx3 numpy array * simtk.unit compatible with simtk.unit.nanometers
            ParmEd positions of the atoms to be moved.
        masses : numpy.array
            numpy.array of particle masses
        Returns
        -------
        center_of_mass: numpy array * simtk.unit compatible with simtk.unit.nanometers
            1x3 numpy.array of the center of mass of the given positions
        """
        coordinates = np.asarray(positions._value, np.float32)
        center_of_mass = parmed.geometry.center_of_mass(coordinates, masses) * positions.unit
        return center_of_mass
    
    def apply(self, thermodynamic_state, sampler_state):
        """Apply a metropolized move to the sampler state.

        Total number of acceptances and proposed move are updated.

        Parameters
        ----------
        thermodynamic_state : openmmtools.states.ThermodynamicState
           The thermodynamic state to use to apply the move.
        sampler_state : openmmtools.states.SamplerState
           The initial sampler state to apply the move to. This is modified.

        """
        timer = Timer()
        benchmark_id = 'Applying {}'.format(self.__class__.__name__ )
        timer.start(benchmark_id)

        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            context_cache = self.context_cache
            
        # Create integrator
        integrator = self._get_integrator(thermodynamic_state)

        # Create context
        context, integrator = context_cache.get_context(thermodynamic_state, integrator)
        self.kT = integrator.kT
        
        # Compute initial energy. We don't need to set velocities to compute the potential.
        # TODO assume sampler_state.potential_energy is the correct potential if not None?
        sampler_state.apply_to_context(context, ignore_velocities=True)
        initial_energy = thermodynamic_state.reduced_potential(context)
        
        masses, totalmass = self.getMasses(self.atom_subset, thermodynamic_state.topology)
        # Handle default and weird cases for atom_subset.
        if self.atom_subset is None:
            atom_subset = slice(None)
        elif not isinstance(self.atom_subset, slice) and len(self.atom_subset) == 1:
            # Slice so that initial_positions (below) will have a 2D shape.
            atom_subset = slice(self.atom_subset[0], self.atom_subset[0]+1)
        else:
            atom_subset = slice(self.atom_subset[0], self.atom_subset[-1]+1)#self.atom_subset
        
        # Store initial positions of the atoms that are moved.
        # We'll use this also to recover in case the move is rejected.
        if isinstance(atom_subset, slice):
            # Numpy array when sliced return a view, they are not copied.
            initial_positions = copy.deepcopy(sampler_state.positions[atom_subset])
        else:
            # This automatically creates a copy.
            initial_positions = sampler_state.positions[atom_subset]
            
        # Calculate the center of mass
        center_of_mass = self.getCenterOfMass(initial_positions, masses)
        
        # Run dynamics
        #NML: Do in 1 steps for debugging
        timer.start("{}: step({})".format(benchmark_id, n_steps))
        rotation_step = int(self.n_steps/2)
        for n in range(self.n_steps+1):
            integrator.step(1)

            step = integrator.getGlobalVariableByName('step')
            alch_lambda = integrator.getGlobalVariableByName('lambda')
            lambda_step = integrator.getGlobalVariableByName('lambda_step')
            protocol_work = integrator.getGlobalVariableByName('protocol_work')

            thermodynamic_state.set_alchemical_variable('lambda', alch_lambda)
            thermodynamic_state.apply_to_context(context)

            #lambda_sterics = context.getParameter('lambda_sterics')
            #lambda_electrostatics = context.getParameter('lambda_electrostatics')

            if n == rotation_step:
                # Propose perturbed positions. Modifying the reference changes the sampler state.
                proposed_positions = self._propose_positions(initial_positions, center_of_mass)
                # Compute the energy of the proposed positions.
                sampler_state.positions[atom_subset] = proposed_positions
                sampler_state.apply_to_context(context, ignore_velocities=True)
                
            # Get the context state for reporting
            context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                     enforcePeriodicBox=thermodynamic_state.is_periodic)
            #print('IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step) 
            #print('\t'+'ContextParameters:', lambda_step, 'Sterics:', lambda_sterics, 'Elec:', lambda_electrostatics)     
            if n % 10 == 0:
                print('NCMC---IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step) 
                
            if step == self.n_steps:
                break
                
        timer.stop("{}: step({})".format(benchmark_id, self.n_steps))

        # Subclasses can read here info from the context to update internal statistics.
        #self._after_integration(context, thermodynamic_state)
 
        # Updated sampler state.
        timer.start("{}: update sampler state".format(benchmark_id))    
        # This is an optimization around the fact that Collective Variables are not a part of the State,
        # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
        # the State.

         # Update everything but the collective variables from the State object
        sampler_state.update_from_context(context_state, ignore_collective_variables=True, ignore_velocities=True)
        # Update only the collective variables from the Context
        sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                          ignore_collective_variables=False)                                          
        timer.stop("{}: update sampler state".format(benchmark_id))
    
        proposed_energy = thermodynamic_state.reduced_potential(context)
        
        self.initial_positions = initial_positions
        self.initial_energy = initial_energy
        self.proposed_positions = proposed_positions
        self.proposed_energy = proposed_energy
        
        # Accept or reject with Metropolis criteria.
        #delta_energy = proposed_energy - initial_energy
        #if (not np.isnan(proposed_energy) and
        #        (delta_energy <= 0.0 or np.random.rand() < np.exp(-delta_energy))):
        #    self.n_accepted += 1
        #else:
            # Restore original positions.
        #    sampler_state.positions[atom_subset] = initial_positions
        #self.n_proposed += 1

        # Print timing information.
        timer.stop(benchmark_id)
        #timer.report_timing()


In [4]:
class ModLangevinDynamicsMove(LangevinDynamicsMove):
    
    def _before_integration(self, context, thermodynamic_state):
        """Execute code after Context creation and before integration."""
        self.initial_energy = thermodynamic_state.reduced_potential(context)
        
    def _after_integration(self, context, thermodynamic_state):
        """Execute code after integration.

        After this point there are no guarantees that the Context will still
        exist, together with its bound integrator and system.
        """
        self.final_energy = thermodynamic_state.reduced_potential(context)

In [5]:
from blues import utils   
# Define parameters
temperature = 300 * unit.kelvin
collision_rate = 1 / unit.picoseconds
timestep = 4.0 * unit.femtoseconds
n_steps = 100
reportInterval = 1000
nIter=100
prmtop = utils.get_data_filename('blues', 'tests/data/eqToluene.prmtop')
inpcrd = utils.get_data_filename('blues', 'tests/data/eqToluene.inpcrd')
tol = parmed.load_file(prmtop, xyz=inpcrd)
tol.system = tol.createSystem(nonbondedMethod=openmm.app.PME,
                             nonbondedCutoff=10*unit.angstrom,
                             constraints=openmm.app.HBonds,
                             hydrogenMass=3.024*unit.dalton,
                             rigidWater=True,removeCMMotion=True,
                             flexibleConstraints=True, splitDihedrals=False
                             )

2019-02-01 17:25:42,437 | INFO : Adding bonds...
2019-02-01 17:25:42,546 | INFO : Adding angles...
2019-02-01 17:25:42,559 | INFO : Adding dihedrals...
2019-02-01 17:25:42,592 | INFO : Adding Ryckaert-Bellemans torsions...
2019-02-01 17:25:42,593 | INFO : Adding Urey-Bradleys...
2019-02-01 17:25:42,593 | INFO : Adding improper torsions...
2019-02-01 17:25:42,594 | INFO : Adding CMAP torsions...
2019-02-01 17:25:42,595 | INFO : Adding trigonal angle terms...
2019-02-01 17:25:42,597 | INFO : Adding out-of-plane bends...
2019-02-01 17:25:42,597 | INFO : Adding pi-torsions...
2019-02-01 17:25:42,598 | INFO : Adding stretch-bends...
2019-02-01 17:25:42,598 | INFO : Adding torsion-torsions...
2019-02-01 17:25:42,599 | INFO : Adding Nonbonded force...


In [6]:
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False,
                                            disable_alchemical_dispersion_correction=True,
                                           alchemical_pme_treatment='direct-space')
alchemical_atom_idx = utils.atomIndexfromTop('LIG',tol.topology)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=alchemical_atom_idx,
                                             annihilate_sterics=False, 
                                             annihilate_electrostatics=True)
alchemical_atoms = list(alchemical_region.alchemical_atoms)
toluene_alchemical_system = factory.create_alchemical_system(
    reference_system=tol.system, alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(toluene_alchemical_system)
# Create our custom State objects
# Need two different Thermodynamic State objects
# Context cache will grab correct thermodynamic state
# Keeping them in sync is in SamplerState.apply to context
# Have apply return accumulated work

alch_thermodynamic_state = ThermodynamicState(system=toluene_alchemical_system, temperature=temperature)
alch_thermodynamic_state = CompoundThermodynamicState(alch_thermodynamic_state, composable_states=[alchemical_state])
alch_thermodynamic_state.alchemical_atoms = alchemical_atoms
alch_thermodynamic_state.topology = tol.topology
thermodynamic_state = ThermodynamicState(system=tol.system, temperature=temperature)
sampler_state = SamplerState(positions=tol.positions)
context_cache = cache.ContextCache(capacity=1)
atom_subset = slice(alchemical_atoms[0], alchemical_atoms[-1]+1)

In [7]:
sampler_state.positions[atom_subset]

Quantity(value=[(22.37, 21.68, 29.77), (22.6, 22.93, 30.42), (23.4, 21.08, 29.01), (23.76, 23.59, 30.25), (24.62, 21.76, 28.95), (24.83, 22.98, 29.58), (26.1, 23.68, 29.4), (21.39, 21.22, 29.86), (21.83, 23.42, 31.02), (23.15, 20.18, 28.46), (23.99, 24.57, 30.68), (25.44, 21.27, 28.41), (25.91, 24.73, 29.15), (26.67, 23.64, 30.34), (26.79, 23.19, 28.72)], unit=angstrom)

In [8]:
langevin_move = ModLangevinDynamicsMove(timestep, collision_rate, n_steps, 
                                     reassign_velocities=True, 
                                    context_cache=context_cache)
langevin_move.apply(thermodynamic_state, sampler_state)

In [9]:
langevin_move.initial_energy

-115874.91915629979

In [10]:
langevin_move.final_energy

-115674.96660299534

In [11]:
sampler_state.positions[atom_subset]

Quantity(value=array([[2.2573688 , 2.18946815, 2.93071961],
       [2.28345394, 2.29781771, 3.00894141],
       [2.3499403 , 2.13557816, 2.8397243 ],
       [2.40530562, 2.37166548, 2.98927736],
       [2.46458769, 2.20837331, 2.82125783],
       [2.4943769 , 2.3219893 , 2.89723492],
       [2.63198304, 2.39439368, 2.8773098 ],
       [2.16232467, 2.13883162, 2.94549394],
       [2.21099114, 2.32580948, 3.0849762 ],
       [2.34072232, 2.04589057, 2.77900457],
       [2.42384171, 2.45197821, 3.06014347],
       [2.52668905, 2.17285943, 2.73941731],
       [2.61486435, 2.50176859, 2.86720133],
       [2.68999529, 2.37634659, 2.96804857],
       [2.68205953, 2.35921407, 2.78687024]]), unit=nanometer)

from blues.reporters import NetCDF4Reporter
with open('test.pdb', 'w') as pdb:
    openmm.app.pdbfile.PDBFile.writeFile(tol.topology, tol.positions, pdb)
filename = 'test.nc'
if os.path.exists(filename):
    os.remove(filename)
else:
    print("Sorry, I can not remove %s file." % filename)
nc_reporter = NetCDF4Reporter(filename, 1)
nc_reporter.reportInterval = reportInterval

In [12]:
ncmc_move = NCMCMove(atom_subset=alchemical_atoms, context_cache=context_cache, n_steps=n_steps)
ncmc_move.apply(alch_thermodynamic_state, sampler_state)

/home/nathanlim/SCRATCH/molssi-project/openmmtools/openmmtools/alchemy.py:344: UserWarning: AlchemicalState.get_alchemical_variable is deprecated. Use AlchemicalState.get_function_variable instead.
  warnings.warn('AlchemicalState.get_alchemical_variable is deprecated. '


NCMC---IntgrParameters: Step: 1.0 PE: -294164.0831375539 kJ/mol Work: 0.875 Lambda:  0.01 Lambda Step:  2.0
NCMC---IntgrParameters: Step: 11.0 PE: -317260.7081375539 kJ/mol Work: 10.5 Lambda:  0.11 Lambda Step:  22.0
NCMC---IntgrParameters: Step: 21.0 PE: -318146.2081375539 kJ/mol Work: 20.25 Lambda:  0.21 Lambda Step:  42.0
NCMC---IntgrParameters: Step: 31.0 PE: -319179.4581375539 kJ/mol Work: 42.125 Lambda:  0.31 Lambda Step:  62.0
NCMC---IntgrParameters: Step: 41.0 PE: -318668.4581375539 kJ/mol Work: 68.875 Lambda:  0.41 Lambda Step:  82.0
Proposing positions...
NCMC---IntgrParameters: Step: 51.0 PE: 34098451.79186244 kJ/mol Work: 90.375 Lambda:  0.51 Lambda Step:  102.0
NCMC---IntgrParameters: Step: 61.0 PE: 169893787.79186246 kJ/mol Work: 29373.25 Lambda:  0.61 Lambda Step:  122.0
NCMC---IntgrParameters: Step: 71.0 PE: 31537197.791862447 kJ/mol Work: 29621.0 Lambda:  0.71 Lambda Step:  142.0
NCMC---IntgrParameters: Step: 81.0 PE: -305726.8331375539 kJ/mol Work: 31302.125 Lambda:  

In [13]:
ncmc_move.initial_positions

Quantity(value=array([[2.2573688 , 2.18946815, 2.93071961],
       [2.28345394, 2.29781771, 3.00894141],
       [2.3499403 , 2.13557816, 2.8397243 ],
       [2.40530562, 2.37166548, 2.98927736],
       [2.46458769, 2.20837331, 2.82125783],
       [2.4943769 , 2.3219893 , 2.89723492],
       [2.63198304, 2.39439368, 2.8773098 ],
       [2.16232467, 2.13883162, 2.94549394],
       [2.21099114, 2.32580948, 3.0849762 ],
       [2.34072232, 2.04589057, 2.77900457],
       [2.42384171, 2.45197821, 3.06014347],
       [2.52668905, 2.17285943, 2.73941731],
       [2.61486435, 2.50176859, 2.86720133],
       [2.68999529, 2.37634659, 2.96804857],
       [2.68205953, 2.35921407, 2.78687024]]), unit=nanometer)

In [14]:
ncmc_move.proposed_positions

Quantity(value=array([[2.34422101, 2.37228898, 2.77158618],
       [2.40758887, 2.4395349 , 2.87159224],
       [2.32386727, 2.23322604, 2.77257768],
       [2.47069422, 2.36569117, 2.97767154],
       [2.38622886, 2.16462601, 2.87351943],
       [2.45493095, 2.22922117, 2.97684413],
       [2.51529831, 2.1446411 , 3.09421897],
       [2.30154033, 2.43127416, 2.69087166],
       [2.4086844 , 2.54819176, 2.86873361],
       [2.26854517, 2.17246441, 2.70142144],
       [2.51243252, 2.42531269, 3.05841134],
       [2.38172344, 2.05655454, 2.86275169],
       [2.6184044 , 2.1760662 , 3.11171747],
       [2.45589031, 2.16773176, 3.18288776],
       [2.51581355, 2.03824386, 3.06964388]]), unit=nanometer)

In [15]:
sampler_state.positions[atom_subset]

Quantity(value=array([[2.33354282, 2.40801311, 2.84397006],
       [2.4434545 , 2.35866785, 2.88867116],
       [2.24809527, 2.31811476, 2.79920912],
       [2.48508835, 2.22656465, 2.91278768],
       [2.2580874 , 2.18139863, 2.82494497],
       [2.39076972, 2.1309762 , 2.87637711],
       [2.42876005, 1.99098396, 2.84996176],
       [2.34140921, 2.5128541 , 2.81636286],
       [2.5302217 , 2.42342973, 2.89831758],
       [2.14911962, 2.33403802, 2.75718641],
       [2.57322145, 2.19465494, 2.96783304],
       [2.17664027, 2.1108253 , 2.81075072],
       [2.37583327, 1.91033983, 2.90114689],
       [2.43365526, 1.98175144, 2.74126291],
       [2.51006627, 1.97459745, 2.92099285]]), unit=nanometer)

In [16]:
# Get alch_PE by using the thermodynamic state from MD 
# with final positions from NCMC simulations

alch_energy = thermodynamic_state.reduced_potential(sampler_state)
alch_energy

-120568.06537377278

In [18]:
correction_factor = (ncmc_move.initial_energy - langevin_move.initial_energy 
                     + alch_energy - ncmc_move.proposed_energy) * (-1.0 / ncmc_move.kT)
correction_factor

Quantity(value=-79.2084584340074, unit=mole/kilojoule)

In [9]:
ncmc_move = NCMCMove(timestep=timestep,
                 collision_rate=collision_rate,
                 n_steps=n_steps,
                 temperature=temperature,
                 n_restart_attempts=5,
                 reassign_velocities=True)
sampler = NCMCSampler(thermodynamic_state, 
                      alch_thermodynamic_state,
                      sampler_state,
                      move=ncmc_move, 
                      platform=None, reporter=nc_reporter, 
                      topology=tol.topology)
sampler.minimize(max_iterations=0)

2019-01-31 17:28:39,581 | DEBUG : LocalEnergyMinimizer: platform is CUDA
2019-01-31 17:28:39,582 | DEBUG : Minimizing with tolerance 1.0 kcal/(A mol) and 0 max. iterations.
-357241.39762897673 kJ/mol


In [10]:
sampler.run(n_iterations=nIter)

................................................................................
MCMC sampler iteration 0


NameError: name 'integrator' is not defined

In [ ]:
traj_nc = mdtraj.load_netcdf('test.nc', top='test.pdb',stride=10)
traj_nc.image_molecules()

In [ ]:
view = nglview.show_mdtraj(traj_nc, gui=True)
#view.camera = 'orthographic'
view.background = 'white'
# clear representations
#view.clear_representations()
# specify residue
view.add_licorice('ligand')
view.add_line('water')

view

In [3]:
class NCMCMove(BaseIntegratorMove):

    def __init__(self,
                 timestep=1.0 * unit.femtosecond,
                 collision_rate=10.0 / unit.picoseconds,
                 n_steps=1000,
                 temperature=300 * unit.kelvin,
                 reassign_velocities=True,
                 **kwargs):
        super(NCMCMove, self).__init__(n_steps=n_steps, **kwargs)
        self.timestep = timestep
        self.temperature = temperature
        self.collision_rate = collision_rate
        self.n_accepted = 0  # Number of accepted steps.
        self.n_proposed = 0  # Number of attempted steps.
        self.random_state = np.random.RandomState(1)

    @property
    def fraction_accepted(self):
        """Ratio between accepted over attempted moves (read-only).

        If the number of attempted steps is 0, this is numpy.NaN.

        """
        if self.n_proposed == 0:
            return np.NaN
        # TODO drop the casting when stop Python2 support
        return float(self.n_accepted) / self.n_proposed

    @property
    def statistics(self):
        """The acceptance statistics as a dictionary."""
        return dict(n_accepted=self.n_accepted, n_proposed=self.n_proposed)

    @statistics.setter
    def statistics(self, value):
        self.n_accepted = value['n_accepted']
        self.n_proposed = value['n_proposed']

    def reset_statistics(self):
        """Reset the internal statistics of number of accepted and attempted moves."""
        self.n_accepted = 0
        self.n_proposed = 0
        
    def _create_dummy_simulation(self, integrator, system, topology):
        """
        Generate a dummy Simulation object because the Reporter
        expects an `openmm.Simulation` object in order to report information
        from it's respective attached integrator/
        """
        return DummySimulation(integrator, system, topology) 
    
    def _before_integration(self, context, integrator, thermodynamic_state):
        """Execute code after Context creation and before integration."""
        
        return self._create_dummy_simulation(integrator,
                                            thermodynamic_state.get_system(), 
                                            thermodynamic_state.topology)
    
    def getMasses(self, atom_subset, topology):
        """
        Returns a list of masses of the specified ligand atoms.
        Parameters
        ----------
        topology: parmed.Topology
            ParmEd topology object containing atoms of the system.
        Returns
        -------
        masses: 1xn numpy.array * simtk.unit.dalton
            array of masses of len(self.atom_indices), denoting
            the masses of the atoms in self.atom_indices
        totalmass: float * simtk.unit.dalton
            The sum of the mass found in masses
        """
        atoms = [ list(topology.atoms())[i] for i in atom_subset]
        masses = unit.Quantity(np.zeros([int(len(atoms)), 1], np.float32), unit.dalton)
        for idx, atom in enumerate(atoms):
            masses[idx] = atom.element._mass
        totalmass = masses.sum()
        return masses, totalmass

    def getCenterOfMass(self, positions, masses):
        """Returns the calculated center of mass of the ligand as a numpy.array
        Parameters
        ----------
        positions: nx3 numpy array * simtk.unit compatible with simtk.unit.nanometers
            ParmEd positions of the atoms to be moved.
        masses : numpy.array
            numpy.array of particle masses
        Returns
        -------
        center_of_mass: numpy array * simtk.unit compatible with simtk.unit.nanometers
            1x3 numpy.array of the center of mass of the given positions
        """
        coordinates = np.asarray(positions._value, np.float32)
        center_of_mass = parmed.geometry.center_of_mass(coordinates, masses) * positions.unit
        return center_of_mass
    
    def rotate(self, context, thermodynamic_state, sampler_state):
        print('Performing rotation...')
        atom_subset = thermodynamic_state.alchemical_atoms
        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                 enforcePeriodicBox=thermodynamic_state.is_periodic)
        all_positions = context_state.getPositions(asNumpy=True)
        sub_positions = all_positions[atom_subset]
        
        masses, totalmass = self.getMasses(atom_subset, thermodynamic_state.topology)
        center_of_mass = self.getCenterOfMass(sub_positions, masses)
        reduced_pos = sub_positions - center_of_mass

        # Define random rotational move on the ligand
        rand_quat = mdtraj.utils.uniform_quaternion(size=None, random_state=self.random_state)
        rand_rotation_matrix = mdtraj.utils.rotation_matrix_from_quaternion(rand_quat)
        #multiply lig coordinates by rot matrix and add back COM translation from origin
        rot_move = np.dot(reduced_pos, rand_rotation_matrix) * sub_positions.unit + center_of_mass

        # Update ligand positions in sampler_state and context
        for index, atomidx in enumerate(atom_subset):
            all_positions[atomidx] = rot_move[index]
        sampler_state.positions = all_positions
        sampler_state.apply_to_context(context, ignore_velocities=True)
      
        return context
    
    def apply(self, thermodynamic_state, sampler_state, integrator, reporter, alch=True, ignore_velocities=True):
        """Propagate the state through the integrator.
        This updates the SamplerState after the integration. It also logs
        benchmarking information through the utils.Timer class.
        Parameters
        ----------
        thermodynamic_state : openmmtools.states.ThermodynamicState
           The thermodynamic state to use to propagate dynamics.
        sampler_state : openmmtools.states.SamplerState
           The sampler state to apply the move to. This is modified.
        See Also
        --------
        openmmtools.utils.Timer
        """
        move_name = self.__class__.__name__  # shortcut
        timer = Timer()

        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            context_cache = self.context_cache

        # Create context.
        timer.start("{}: Context request".format(move_name))
        context, integrator = context_cache.get_context(thermodynamic_state, integrator)
        
        print(integrator)
        timer.stop("{}: Context request".format(move_name))
        logger.debug("{}: {} \t Context obtained, platform is {}".format(
            move_name, integrator, context.getPlatform().getName()))
        
        
        if alch:
            rotation_step = int(n_steps/2)
        
        
        # Perform the integration.
        for attempt_counter in range(self.n_restart_attempts + 1):
            
            if not alch:
                # If we reassign velocities, we can ignore the ones in sampler_state.
                sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
                if self.reassign_velocities:
                    context.setVelocitiesToTemperature(thermodynamic_state.temperature)
            if alch:
                # If we reassign velocities, we can ignore the ones in sampler_state.
                sampler_state.apply_to_context(context, ignore_velocities=False)
  
            # Subclasses may implement _before_integration().
            simulation = self._before_integration(context, integrator, thermodynamic_state)
            init_context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                         enforcePeriodicBox=thermodynamic_state.is_periodic)
            try:
                # Run dynamics.
                timer.start("{}: step({})".format(move_name, self.n_steps))
                
                #NML: Do in 1 steps for debugging
                for n in range(self.n_steps+1):
                    integrator.step(1)
                 
                    if alch:
                        step = integrator.getGlobalVariableByName('step')
                        alch_lambda = integrator.getGlobalVariableByName('lambda')
                        lambda_step = integrator.getGlobalVariableByName('lambda_step')
                        protocol_work = integrator.getGlobalVariableByName('protocol_work')

                        thermodynamic_state.set_alchemical_variable('lambda', alch_lambda)
                        thermodynamic_state.apply_to_context(context)

                        #lambda_sterics = context.getParameter('lambda_sterics')
                        #lambda_electrostatics = context.getParameter('lambda_electrostatics')
                        
                        if n == rotation_step:
                            context = self.rotate(context, thermodynamic_state, sampler_state)
                            
                        # Get the context state for reporting
                        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)
                        #print('IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step) 
                        #print('\t'+'ContextParameters:', lambda_step, 'Sterics:', lambda_sterics, 'Elec:', lambda_electrostatics)     
                        
                        if step == n_steps:
                            break
                
                    else:
                        # Get the context state for reporting
                        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)
                        #print('Pos:', context_state.getPositions()[0])
                        #print('Step:', n, 'PE:', context_state.getPotentialEnergy())
                        
                    if not alch:
                        if n % reporter.reportInterval == 0:
                            print('MD Step:', n, 'PE:', context_state.getPotentialEnergy())
                            reporter.report(simulation, context_state)
                    else:
                        if n % reporter.reportInterval == 0:
                            print('NCMC---IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step) 
                
            except Exception as e:
                print(e)
                
                # Catches particle positions becoming nan during integration.
                restart = True
            else:
                timer.stop("{}: step({})".format(move_name, self.n_steps))

                # We get also velocities here even if we don't need them because we
                # will recycle this State to update the sampler state object. This
                # way we won't need a second call to Context.getState().
                context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)

                # Check for NaNs in energies.
                potential_energy = context_state.getPotentialEnergy()
                restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

            # Restart the move if we found NaNs.
            if restart:
                err_msg = ('Potential energy is NaN after {} attempts of integration '
                           'with move {}'.format(attempt_counter, self.__class__.__name__))

                # If we are on our last chance before crash, try to re-initialize context
                if attempt_counter == self.n_restart_attempts - 1:
                    logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                    context.reinitialize()
                    if alch:
                        sampler_state.apply_to_context(context, ignore_velocities=False)
                    if not alch:
                        sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
                    thermodynamic_state.apply_to_context(context)
                # If we have hit the number of restart attempts, raise an exception.
                elif attempt_counter == self.n_restart_attempts:
                    # Restore the context to the state right before the integration.
                    if alch:
                        sampler_state.apply_to_context(context, ignore_velocities=False)
                    if not alch:
                        sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
                    logger.error(err_msg)
                    print(IntegratorMoveError(err_msg, self, context))
                    break
                else:
                    logger.warning(err_msg + ' Attempting a restart...')
            else:
                break

        # Subclasses can read here info from the context to update internal statistics.
        self._after_integration(context, thermodynamic_state)

        # Updated sampler state.
        timer.start("{}: update sampler state".format(move_name))    
        # This is an optimization around the fact that Collective Variables are not a part of the State,
        # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
        # the State.
        if not alch:
            # Update everything but the collective variables from the State object
            sampler_state.update_from_context(context_state, ignore_collective_variables=True)
            # Update only the collective variables from the Context
            sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                              ignore_collective_variables=False)
        if alch:
             # Update everything but the collective variables from the State object
            sampler_state.update_from_context(context_state, ignore_collective_variables=True, ignore_velocities=True)
            # Update only the collective variables from the Context
            sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                              ignore_collective_variables=False)
                                              
        timer.stop("{}: update sampler state".format(move_name))
        final_context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                    enforcePeriodicBox=thermodynamic_state.is_periodic)


        #timer.report_timing()
        return context, init_context_state, final_context_state
    
    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        return LangevinIntegrator(temperature=thermodynamic_state.temperature,
                                          collision_rate=self.collision_rate,
                                          timestep=self.timestep)

    def __getstate__(self):
        if self.context_cache is None:
            context_cache_serialized = None
        else:
            context_cache_serialized = utils.serialize(self.context_cache)
        return dict(n_steps=self.n_steps, context_cache=context_cache_serialized,
                    reassign_velocities=self.reassign_velocities,
                    n_restart_attempts=self.n_restart_attempts)

    def __setstate__(self, serialization):
        self.n_steps = serialization['n_steps']
        self.reassign_velocities = serialization['reassign_velocities']
        self.n_restart_attempts = serialization['n_restart_attempts']
        if serialization['context_cache'] is None:
            self.context_cache = None
        else:
            self.context_cache = utils.deserialize(serialization['context_cache'])


    def _after_integration(self, context, thermodynamic_state):
        """Implement BaseIntegratorMove._after_integration()."""
        integrator = context.getIntegrator()
        try:
            # Accumulate acceptance statistics.
            ghmc_global_variables = {integrator.getGlobalVariableName(index): index
                                 for index in range(integrator.getNumGlobalVariables())}
            n_accepted = integrator.getGlobalVariable(ghmc_global_variables['naccept'])
            n_proposed = integrator.getGlobalVariable(ghmc_global_variables['ntrials'])
            self.n_accepted += n_accepted
            self.n_proposed += n_proposed
        except Exception as e:
            print(e)
            pass


In [4]:
class NCMCSampler(object):


    def __init__(self, thermodynamic_state=None, alch_thermodynamic_state=None, 
                 sampler_state=None, move=None, platform=None, reporter=None, 
                 pdbfile=None, topology=None):
        """
        Create an MCMC sampler.
        Parameters
        ----------
        thermodynamic_state : ThermodynamicState
            The thermodynamic state to simulate
        sampler_state : SamplerState
            The initial sampler state to simulate from.
        platform : simtk.openmm.Platform, optional, default=None
            If specified, this platform will be used
        ncfile : netCDF4.Dataset, optional, default=None
            NetCDF storage file.
        """

        if alch_thermodynamic_state is None:
            raise Exception("'alch_thermodynamic_state' must be specified")
        if thermodynamic_state is None:
            raise Exception("'thermodynamic_state' must be specified")
        if sampler_state is None:
            raise Exception("'sampler_state' must be specified")
  
        # Make a deep copy of the state so that initial state is unchanged.    
        self.alch_thermodynamic_state = copy.deepcopy(alch_thermodynamic_state)
        self.thermodynamic_state = copy.deepcopy(thermodynamic_state)
        self.sampler_state = copy.deepcopy(sampler_state)
        self.move = move
        
        #NML: Attach topology to thermodynamic_states
        self.alch_thermodynamic_state.topology = topology
        self.thermodynamic_state.topology = topology    
        
        # Initialize
        self.accept = 0 
        self.reject = 0
        self.iteration = 0
        
        # For GHMC / Langevin integrator
        self.collision_rate = move.collision_rate
        self.timestep = move.timestep
        self.n_steps = move.n_steps  # number of steps per update
        self.verbose = True
        self.platform = platform

        # For writing trajectory files
        self.reporter = reporter
        self._timing = dict()
        
    def _get_alch_integrator(self, alch_thermodynamic_state):
        return AlchemicalExternalLangevinIntegrator(
            alchemical_functions={ 'lambda_sterics':
                'min(1, (1/0.3)*abs(lambda-0.5))',
                'lambda_electrostatics':
                'step(0.2-lambda) - 1/0.2*lambda*step(0.2-lambda) + 1/0.2*(lambda-0.8)*step(lambda-0.8)'},
            splitting="H V R O R V H",
            temperature=alch_thermodynamic_state.temperature,
            nsteps_neq=self.n_steps,
            timestep=self.timestep,
            nprop=1,
            prop_lambda=0.3)
    
    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        return LangevinIntegrator(temperature=thermodynamic_state.temperature,
                                          collision_rate=self.collision_rate,
                                  timestep=self.timestep)
    @classmethod
    def setContextFromState(cls, context, state, box=True, positions=True, velocities=True):
        """Update a given Context from the given State.
        Parameters
        ----------
        context : openmm.Context
            The Context to be updated from the given State.
        state : openmm.State
            The current state (box_vectors, positions, velocities) of the
            Simulation to update the given context.
        Returns
        -------
        context : openmm.Context
            The updated Context whose box_vectors, positions, and velocities
            have been updated.
        """
        # Replace ncmc data from the md context
        if box:
            context.setPeriodicBoxVectors(*state.getPeriodicBoxVectors())
        if positions:
            context.setPositions(state.getPositions(asNumpy=True))
        if velocities:
            context.setVelocities(state.getVelocities(asNumpy=True))
        return context     
    
    def _computeAlchemicalCorrection(self, md_context, md_init_context_state, ncmc_init_context_state, ncmc_final_context_state, kT):
        """Computes the alchemical correction term from switching between the NCMC
        and MD potentials."""
        # Retrieve the MD/NCMC PE before the proposed move.
        md_init_PE = md_init_context_state.getPotentialEnergy()
        ncmc_init_PE = ncmc_init_context_state.getPotentialEnergy()
        
        # Retreive the NCMC state after the proposed move.
        ncmc_final_PE = ncmc_final_context_state.getPotentialEnergy()
        md_final_context_state = md_context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                enforcePeriodicBox=self.thermodynamic_state.is_periodic)
        
        # Set the box_vectors and positions in the alchemical simulation to after the proposed move.
        md_context = self.setContextFromState(md_context, ncmc_final_context_state, velocities=False)
        
        #self._ncmc_sim.context._integrator.kT
        # Retrieve potential_energy for alch correction
        alch_PE = md_context.getState(getEnergy=True).getPotentialEnergy()
        correction_factor = (ncmc_init_PE - md_init_PE + alch_PE - ncmc_final_PE) * (
            -1.0 / kT)
        
        md_context = self.setContextFromState(md_context, md_final_context_state)
        
        return correction_factor  
    
    def update(self):
        """
        Update the sampler with one step of sampling.
        """
        if self.verbose:
            print("." * 80)
            print("MCMC sampler iteration %d" % self.iteration)
            
        initial_time = time.time()
        
        md_context, md_init_context_state, md_final_context_state = self.move.apply(self.thermodynamic_state,
                           self.sampler_state,
                           integrator,
                           self.reporter,
                           alch=False)     

        alch_integrator = self._get_alch_integrator(self.alch_thermodynamic_state)  
        ncmc_context, ncmc_init_context_state, ncmc_final_context_state = self.move.apply(self.alch_thermodynamic_state,
                                       self.sampler_state,
                                       alch_integrator,
                                       self.reporter,
                                       alch=True,
                                        ignore_velocities=True)     

        work_ncmc = ncmc_context._integrator.getLogAcceptanceProbability(ncmc_context)
        randnum = math.log(np.random.random())
        # Compute correction if work_ncmc is not NaN
        if not np.isnan(work_ncmc):
            correction_factor = self._computeAlchemicalCorrection(md_context, md_init_context_state, 
                                ncmc_init_context_state, ncmc_final_context_state, 
                                ncmc_context._integrator.kT)
            logger.debug(
                'NCMCLogAcceptanceProbability = %.6f + Alchemical Correction = %.6f' % (work_ncmc, correction_factor))
            work_ncmc = work_ncmc + correction_factor

        if work_ncmc > randnum:
            self.accept += 1
            logger.info('NCMC MOVE ACCEPTED: work_ncmc {} > randnum {}'.format(work_ncmc, randnum))
       
            # If accept move, sync NCMC state to MD context
            md_context = self.setContextFromState(md_context, ncmc_final_context_state, velocities=False)
            #self.sampler_state.update_from_context(md_context)
        else:
            self.reject += 1
            logger.info('NCMC MOVE REJECTED: work_ncmc {} < {}'.format(work_ncmc, randnum))

            # If reject move, do nothing,
            # NCMC simulation be updated from MD Simulation next iteration.
            md_context = self.setContextFromState(md_context, md_init_context_state)
            #self.sampler_state.update_from_context(md_context)
            md_final_context_state = md_context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                enforcePeriodicBox=self.thermodynamic_state.is_periodic)
            # Potential energy should be from last MD step in the previous iteration
            md_PE = md_final_context_state.getPotentialEnergy()
            if not math.isclose(md_init_context_state.getPotentialEnergy()._value, md_PE._value, rel_tol=float('1e-%s' % rtol)):
                logger.error(
                    'Last MD potential energy %s != Current MD potential energy %s. Potential energy should match the prior state.'
                    % (md_init_context_state.getPotentialEnergy(), md_PE))
                sys.exit(1)
        
#         # Take some steps with regular LangevinDynamics
#         # Create integrator.
#         md_context = self.move.apply(self.thermodynamic_state,
#                                    self.sampler_state,
#                                     md_context,
#                                    integrator,
#                                    self.reporter,
#                                     alch=False)     

        # Update sampler state.
        self.sampler_state.update_from_context(md_context)
        
        final_time = time.time()
        elapsed_time = final_time - initial_time
        self._timing['sample positions'] = elapsed_time 
        
        if self.verbose:
            final_energy = md_context.getState(getEnergy=True).getPotentialEnergy() * self.thermodynamic_state.beta
            print('Final energy is %12.3f kT' % (final_energy))
            print('elapsed time %8.3f s' % elapsed_time)

    
        # Increment iteration count
        self.iteration += 1

        if self.verbose:
            print("." * 80)

    def run(self, n_iterations=1):
        """
        Run the sampler for the specified number of iterations
        Parameters
        ----------
        niterations : int, optional, default=1
            Number of iterations to run the sampler for.
        """
        for iteration in range(n_iterations):
            self.update()
                   
    def minimize(self,
                 tolerance=1.0 * unit.kilocalories_per_mole / unit.angstroms,
                 max_iterations=100,
                 context_cache=None):
        """Minimize the current configuration.

        Parameters
        ----------
        tolerance : simtk.unit.Quantity, optional
            Tolerance to use for minimization termination criterion (units of
            energy/(mole*distance), default is 1*kilocalories_per_mole/angstroms).
        max_iterations : int, optional
            Maximum number of iterations to use for minimization. If 0, the minimization
            will continue until convergence (default is 100).
        context_cache : openmmtools.cache.ContextCache, optional
            The ContextCache to use for Context creation. If None, the global cache
            openmmtools.cache.global_context_cache is used (default is None).

        """
        if context_cache is None:
            context_cache = cache.global_context_cache
        timer = Timer()

        # Use LocalEnergyMinimizer
        timer.start("Context request")
        integrator = openmm.VerletIntegrator(1.0*unit.femtosecond)
        context, integrator = context_cache.get_context(self.thermodynamic_state, integrator)
        self.sampler_state.apply_to_context(context)
        logger.debug("LocalEnergyMinimizer: platform is %s" % context.getPlatform().getName())
        logger.debug("Minimizing with tolerance %s and %d max. iterations." % (tolerance, max_iterations))
        timer.stop("Context request")

        timer.start("LocalEnergyMinimizer minimize")
        openmm.LocalEnergyMinimizer.minimize(context, tolerance, max_iterations)
        timer.stop("LocalEnergyMinimizer minimize")

        # Retrieve data.
        self.sampler_state.update_from_context(context)
        
        context_state = context.getState(
            getPositions=True,
            getVelocities=True,
            getEnergy=True,
            enforcePeriodicBox=self.thermodynamic_state.is_periodic)
        potential_energy = context_state.getPotentialEnergy()
        print(potential_energy)


        #timer.report_timing()